In [1]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as sci_plt

from pprint import pprint
from sklearn.preprocessing import StandardScaler
import yfinance as yf
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)


In [2]:
ativos = ['ITUB4.SA', 'BBDC4.SA', 'ABEV3.SA', 'PETR3.SA', 'VALE3.SA']
num_ativos = len(ativos)
csv_file_path = "data/stock_data.csv"

if not pathlib.Path(csv_file_path).exists():
    stock_data = yf.download(ativos, start="2020-01-01", end="2025-01-01", group_by="ticker")
    stock_data.to_csv(csv_file_path, index=True)
else:
    stock_data = pd.read_csv(csv_file_path, header=[0, 1], index_col=0)

price_data_frame = stock_data.xs('Adj Close', axis=1, level=1)

price_data_frame.head()



Ticker,ABEV3.SA,PETR3.SA,ITUB4.SA,VALE3.SA,BBDC4.SA
Date,,,,,
2020-01-02,16.175148,11.703511,31.228830,36.413361,22.384174
2020-01-03,15.947685,11.414492,30.900372,36.145264,22.395496
2020-01-06,16.023504,11.785577,30.440514,35.930790,21.995480
2020-01-07,16.057205,11.625012,29.734314,36.192177,21.613344
2020-01-08,15.981383,11.435903,29.249830,36.198883,21.279007


"How do we measure risk?". Well, this can lead you down a rabbit hole because there are many ways to measure risk and some are more appropriate than others depending on the situation at hand. In our case, we will use a very popular metric, the `Sharpe Ratio`. The Sharpe Ratio is used as a measure for calculating risk-adjusted return and has been the industry standard for such calculations. The Sharpe Ratio allows us to quantify the relationship between the average return earned in excess of the risk-free rate per unit of volatility or total risk.

Mathematically, we define the Sharpe Ratio as the following:

$$
\text{Sharpe Ratio} = \frac{(R_p - R_f)}{\sigma_p}
$$

Where:

$$
\begin{align}
R_p & = \text{Return of Portfolio} \\
R_f & = \text{Risk-Free Rate} \\
\sigma_p & = \text{Standard Deviation of Portfolio's Excess Return} \\
\end{align}
$$

To calculate the expected returns, we use the following formula:

$$
R_p = (w_{1}r_{1}) + (w_{2}r_{2}) + \cdots + (w_{n}r_{n})
$$

Where:

$$
\begin{align}
r_{i} & = \text{Return of Security i} \\
w_{i} & = \text{Weight of Security i} \\
\end{align}
$$

To calculate the standard deviation of the portfolio, we use the following formula:

$$
\sigma_p = \sqrt{(w_{i}^2 \sigma_i^2) + (w_{j}^2 \sigma_j^2) + (2w_{j}w_{i} p_{i,j} \sigma_i \sigma_j)}
$$

Where:

$$
\begin{align}
\sigma_{i} & = \text{Standard Deviation of Returns for Security i} \\
w_{i} & = \text{Weight of Security i} \\
p_{i,j} & = \text{Correlation Coefficient between the returns of asset i and asset j} \\
\end{align}
$$


In [7]:
log_return = np.log(1+price_data_frame.pct_change())

# Pesos aleatórios
pesos_aleatorios = np.array(np.random.random(num_ativos))

# Rebalanceamento dos pesos para que a soma seja 1
pesos_aleatorios_rebal = pesos_aleatorios / np.sum(pesos_aleatorios)

# Retorno esperado, por ano
retorno_esperado = np.sum((log_return.mean() * pesos_aleatorios_rebal) * 252)

# Volatilidade esperada, por ano
volatilidade_esperada = np.sqrt(np.dot(pesos_aleatorios_rebal.T, np.dot(log_return.cov() * 252, pesos_aleatorios_rebal)))

# Calcular o Sharpe Ratio
sharpe_ratio = retorno_esperado / volatilidade_esperada

In [8]:
# Put the weights into a data frame to see them better.
weights_df = pd.DataFrame(data={
'random_weights': pesos_aleatorios,
'rebalance_weights': pesos_aleatorios_rebal
})
print('')
print('='*80)
print('PORTFOLIO WEIGHTS:')
print('-'*80)
print(weights_df)
print('-'*80)

# Do the same with the other metrics.
metrics_df = pd.DataFrame(data={
    'Expected Portfolio Returns': retorno_esperado,
    'Expected Portfolio Volatility': volatilidade_esperada,
    'Portfolio Sharpe Ratio': sharpe_ratio
}, index=[0])

print('')
print('='*80)
print('PORTFOLIO METRICS:')
print('-'*80)
print(metrics_df)
print('-'*80)


PORTFOLIO WEIGHTS:
--------------------------------------------------------------------------------
   random_weights  rebalance_weights
0        0.689263           0.186857
1        0.669513           0.181502
2        0.729076           0.197650
3        0.678401           0.183912
4        0.922473           0.250079
--------------------------------------------------------------------------------

PORTFOLIO METRICS:
--------------------------------------------------------------------------------
   Expected Portfolio Returns  Expected Portfolio Volatility  Portfolio Sharpe Ratio
0                    0.029323                       0.294314                0.099631
--------------------------------------------------------------------------------
